Creating a Simple Tensor with Random Items

In [30]:
import numpy as np

In [46]:
# scientic notation is suppressed by below code
# meaning the decimal palce will stay fixed and conversion 
# to exponential(power of 10-Scientific Notation) form will not be there, this will help us analyse the results better

np.set_printoptions(suppress=True)

In [54]:
#generating a random floating point array
np.random.seed(42)
params = np.random.uniform(low=-50,high=150,size=20)
params

array([ 24.90802377, 140.14286128,  96.39878836,  69.73169684,
       -18.79627191, -18.80109593, -38.38327757, 123.23522915,
        70.22300235,  91.61451556, -45.88310114, 143.98197043,
       116.48852816,  -7.53217786, -13.63500656, -13.31909803,
        10.84844859,  54.95128633,  36.38900373,   8.24582804])

In [55]:
print(params.max())
print(params.min())

143.98197043239887
-45.883101140839514


In [56]:
# adjusting the most important numbers in such a way that we can easily observe the effect of quantization 
params[0] = params.max()-1
params[1] = params.min()-1
params[2] = 0


In [51]:
params

array([142.98197043, -46.88310114,   0.        ,  69.73169684,
       -18.79627191, -18.80109593, -38.38327757, 123.23522915,
        70.22300235,  91.61451556, -45.88310114, 143.98197043,
       116.48852816,  -7.53217786, -13.63500656, -13.31909803,
        10.84844859,  54.95128633,  36.38900373,   8.24582804])

Defining the quantization menthods

In [ ]:
# clamp function
def clamp(params_q: np.array, lower_bound:int, upper_bound: int) -> np.array:
    params_q[params_q<lower_bound] = lower_bound
    params[params_q>upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array,float,int]:
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha-beta)/(2**bits-1)
    zero = -1*np.round(beta/scale)
    lower_bound = 0
    upper_bound = 2**bits-1
    quantized = clamp(np.round(params/scale)+zero,lower_bound,upper_bound).astype(np.int32)
    # we convert to int32 as np.round even though converts the values to whole numbers but returns float64 datatype by default
    return quantized, scale, zero

def asymmetric_dequantization(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q-zero)*scale

def symmetric_dequantization(params_q: np.array, scale: float) -> np.array:
    return params_q*scale

def symmetric_quantization(params:np.array,bits:int)->tuple[np.array,float]:
    alpha = np.max(abs(params))
    scale = alpha/(2**(bits-1)-1)
    lower_bound = -(2**(bits-1)-1)
    upper_bound = 2**(bits-1)-1
    quantize = clamp(np.round(params/scale),lower_bound,upper_bound).astype(np.int32)
    return quantize,scale

def quantization_error(params:np.array, params_q:np.array):
    # mean squared error
    return np.mean((params-params_q)**2)


In [58]:
asymmetric_q,asymmetric_scale, asymmetric_quantization_zero = asymmetric_quantization(params,8)
symmetric_q,symmetric_scale = symmetric_quantization(params,8)

In [59]:
print('Original:')
print(params)
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_quantization_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)


Original:
[127.         -46.88310114   0.         127.         -18.79627191
 -18.80109593 -38.38327757 127.         127.         127.
 -45.88310114 127.         127.          -7.53217786 -13.63500656
 -13.31909803  10.84844859 127.         127.           8.24582804]

Asymmetric scale: 0.7484904767577976, zero: 63.0
[254   0  63 156  38  38  12 228 157 185   2 255 219  53  45  45  77 136
 112  74]

Symmetric scale: 0.12485914400311615
[1145 -129    0  558 -129 -129 -129  987  562  734 -129 1153  933  -60
 -109 -107   87  440  291   66]


In [40]:
symmetric_q

array([-128,  545,    0, -128, -128, -128, -128, -128,  -34,  108, -128,
       -128, -128,  413,  529, -128, -128,  261,  212,  355])

In [41]:
# performing dequantization

params_deq_asymmetric = asymmetric_dequantization(asymmetric_q, asymmetric_scale, asymmetric_quantization_zero)
params_deq_symmetric = symmetric_dequantization(symmetric_q,symmetric_scale)

In [42]:
print('Original:')
print(params)
print('')
print(f'Dequantize Asymmetric: {params_deq_asymmetric}')
print('')
print(f'Dequantize Symmetric: {params_deq_symmetric}')


Original:
[118.73314904 127.           0.          65.32298924  82.39966922
  77.8322529   22.56922638 119.73314904   2.22533658  -6.98146009
  36.12508686  69.02070893  34.54136323 127.         127.
  33.95895632  73.76921973 127.         127.         127.        ]

Dequantize Asymmetric: [118.46726172 -35.23641631   0.          65.61263726  82.62332099
  77.76312564  22.47840351 119.68231056   2.43009768  -6.68276861
  35.84394073  69.25778377  34.62889189 -26.73107444 -34.02136747
  34.02136747  73.51045471 -17.01068373 -13.36553722 -23.08592792]

Dequantize Symmetric: [  8.26685096 -35.19870136  -0.           8.26685096   8.26685096
   8.26685096   8.26685096   8.26685096   2.19588229  -6.9751555
   8.26685096   8.26685096   8.26685096 -26.67351131 -34.16534499
   8.26685096   8.26685096 -16.85662579 -13.69197191 -22.92759447]


In [44]:
# quantization error
print(f'{"Asymmetric error: "}{np.round(quantization_error(params,params_deq_asymmetric),2)}')
print(f'{"Symmetric error: "}{np.round(quantization_error(params,params_deq_symmetric),2)}')

Asymmetric error: 6942.48
Symmetric error: 9369.81
